# Теория вероятностей на примере датасета Credit Risk

## Описание набора данных

Используется датасет **Credit Risk**, содержащий информацию
о заемщиках и статусе выданных кредитов.

### Назначение датасета
Изучение факторов, влияющих на вероятность дефолта по кредиту.

### Признаки:
- **person_age** — возраст заемщика
- **person_income** — годовой доход
- **person_home_ownership** — тип владения жильём
- **person_emp_length** — стаж работы
- **loan_intent** — цель кредита
- **loan_grade** — кредитный рейтинг
- **loan_amnt** — сумма кредита
- **loan_int_rate** — процентная ставка
- **loan_status** — статус кредита (1 — дефолт, 0 — без дефолта)
- **loan_percent_income** — доля дохода на кредит
- **cb_person_default_on_file** — были ли дефолты ранее
- **cb_person_cred_hist_length** — длина кредитной истории


In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("credit_risk_dataset.csv")
df.head()


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


## Формирование событий

Введем бинарное целевое событие:

- **A (Default)** — заемщик допустил дефолт по кредиту  

Считаем, что событие A произошло, если:

$
loan\_status = 1
$

Введем условное событие:

- **B (Previous Default)** — заемщик имел дефолты ранее


In [2]:
df["Default"] = df["loan_status"]

df["Previous Default"] = df["cb_person_default_on_file"].map({
    "Y": 1,
    "N": 0
})

df.head()


,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,Default,Previous Default
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3,1,1
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2,0,0
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3,1,0
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2,1,0
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4,1,1


## Таблица сопряжённости

Рассмотрим зависимость дефолта
от наличия дефолтов в кредитной истории.


In [3]:
table = pd.crosstab(
    df["Previous Default"],
    df["Default"],
    margins=True
)

table


Default,0,1,All
Previous Default,,,
0,21900,4936,26836
1,3573,2172,5745
All,25473,7108,32581


## Определение событий

Обозначим:

- **A** — дефолт по кредиту
- **B** — наличие дефолтов в прошлом

По таблице сопряжённости определим:
- $P(A)$
- $P(B)$
- $P(A \cap B)$


In [4]:
N = table.loc["All", "All"]

P_A = table.loc["All", 1] / N
P_B = table.loc[1, "All"] / N
P_A_and_B = table.loc[1, 1] / N

P_A, P_B, P_A_and_B


(np.float64(0.21816396059052823),
 np.float64(0.17632976274515821),
 np.float64(0.06666462048433136))

## Условная вероятность

$
P(A|B) = \frac{P(A \cap B)}{P(B)}
$

Интерпретация:
вероятность дефолта при наличии дефолтов в прошлом.


In [5]:
P_A_given_B = P_A_and_B / P_B
P_A_given_B


np.float64(0.3780678851174935)

## Условная вероятность через среднее значение

Вероятность дефолта при условии наличия дефолтов
можно вычислить как среднее значение бинарного признака.


In [6]:
P_A_given_B_mean = df[df["Previous Default"] == 1]["Default"].mean()
P_A_given_B_mean


np.float64(0.37806788511749345)

## Формула полной вероятности

Пусть событие A — дефолт по кредиту.

Гипотезы:
- $B_1$ — заемщик имел дефолты ранее
- $B_2$ — заемщик не имел дефолтов ранее

Тогда:

$
P(A) = P(B_1)P(A|B_1) + P(B_2)P(A|B_2)
$


In [7]:
P_B1 = (df["Previous Default"] == 1).mean()
P_B2 = (df["Previous Default"] == 0).mean()

P_A_given_B1 = df[df["Previous Default"] == 1]["Default"].mean()
P_A_given_B2 = df[df["Previous Default"] == 0]["Default"].mean()

P_A_full = P_B1 * P_A_given_B1 + P_B2 * P_A_given_B2

P_A, P_A_full


(np.float64(0.21816396059052823), np.float64(0.21816396059052823))

## Формула Байеса

$
P(B|A) = \frac{P(A|B)P(B)}{P(A)}
$

Интерпретация:
вероятность того, что заемщик имел дефолты ранее
при условии, что текущий кредит оказался дефолтным.


In [8]:
P_B_given_A = (P_A_given_B * P_B) / P_A
P_B_given_A


np.float64(0.3055711873944851)

## Проверка формулы Байеса через данные

Вероятность события B при условии A
можно вычислить напрямую как среднее значение.


In [9]:
P_B_given_A_mean = df[df["Default"] == 1]["Previous Default"].mean()
P_B_given_A_mean


np.float64(0.3055711873944851)

## Выводы

В работе было продемонстрировано применение:

- условных вероятностей
- формулы полной вероятности
- формулы Байеса

на реальном наборе данных Credit Risk.

Все вероятности были рассчитаны:
- вручную (через таблицы сопряжённости)
- автоматически (через средние значения бинарных признаков)

Полученные результаты совпадают,
что подтверждает корректность вычислений.
